<h1> Minimizando o risco de uma Carteira de Investimentos com otimização linear </h1>

<img src="https://media-exp1.licdn.com/dms/image/C4D12AQH4b_B99E8plQ/article-cover_image-shrink_720_1280/0/1559514788836?e=1629936000&v=beta&t=UOSOnwThfNUoyLrcYNW__SmXREmruEIGWh6pEY0WiuY">

<p>As decisões financeiras na prática não são tomadas em ambiente de total certeza com relação a seus resultados. Por essas decisões estarem fundamentalmente voltadas para o futuro, é imprescindível que se introduza a variável incerteza (risco) como um dos mais significativos aspectos do estudo das finanças corporativas. Saber mensurar corretamente o risco e retorno de um negócio, ou seja, saber exatamente com o que está lidando, é uma das principais habilidades que um empreendedor ou um gestor de fundos precisa ter para obter sucesso.</p>

<p> Como então mensuramos o risco? Primeiro precisamos definir uma medida de risco. De maneira geral, medimos o risco pelo quanto de capital precisamos adicionar à nossa posição de risco para termos uma posição aceitável. Por exemplo, suponha que você tenha 1000 reais para investir no mercado de ações e quer ganhar 1 milhão de reais em três anos. O grau de incerteza de obter esse retorno seria ridiculamente alto, (mas teoricamente seria possível, bastando investir no início do dia, na ação que vai dar maior retorno no final dia, e depois trocando para a ação que vai dar maior retorno no final do dia seguinte, sucessivamente) e deveríamos adicionar um capital bastante expressivo nesse investimento para obtermos uma posição aceitável de risco.</p>

<p> Existem várias maneiras de medir um risco de uma carteira de investimentos. Vamos avaliar neste artigo as três mais importantes: Modelo de Markowitz, Valor em Risco (VaR) e Valor em Risco Condicional (CVaR). Mas antes disso, vamos entender o que é ter uma carteira "coerente". A noção de coerência foi introduzida por Artzner et al e atualmente, é um conceito fundamental relacionado à aceitabilidade de uma medida de risco. A literatura introduz um número de propriedades que são usadas para determinar um medida de risco. As propriedades mais importantes para a medida de risco são: </p>

<ol>
    <li><b>Invariância à translação </b>: Se adicionarmos ou subtrairmos uma quantidade certa de nossa carteira, a medida de risco aumenta ou diminui. Matematicamente, se $A$ é o quanto queremos adicionar aos ganhos da carteira, $X$ e $p$ a nossa medida de risco, então $p(X+A)=p(X)+A$. Veja que, se trocarmos instrumentos de renda varável e alocarmos em renda fixa (nessa caso, $A$ é negativo) a carteira diminui o risco no mesmo montante.</li>
    <li><b>Subaditividade</b>: Na minha opinião, é a propriedade mais importante ao se avaliar uma medida de risco. Esta medida é intimamente relacionada com o efeito da diversificação do portfólio. A medida do risco total da carteira (conjunto de ativos) é menor ou igual que a medida do risco da soma individual dos ativos da carteira ($p(X_{1})+p(X_{2})<p(X_{1}+X_{2})$). É o princípio da Teoria Moderno do Portfólio, do grande Markowitz. E para quem gosta de avaliar o desempenho de uma carteira usando o VaR, desculpe decepcioná-los, mais <b>o VaR falha na subaditivade</b>. Isso significa que, minimizar o VaR não garante que você vá diversificar os investimentos da carteira considerada.</li>
    <li><b>Monotonicidade</b>:  Se os ganhos na carteira $X$ são menores que os da carteira $Y$ para todos os cenários possíveis, então o risco na carteira $X$ é menor que na carteira $Y$ (Se $X_{1}<X_{2}$, então $p(X_{1})<p(X_{2})$). Claro, num portfólio devidamente otimizado, se você quiser arriscar mais, espera que sua possibilidade de ganho seja maior. Se você é fã da Teoria de Markowitz, não vai ser fã dessa propriedade, pois infelizmente Markowitz falha na monotonicidade :(.</li>
    <li><b>Homogeneidade Positiva</b>: Ao aumentar o tamanho de cada posição da carteira o risco da carteira aumenta em igual proporção ($p(bX)=bp(X)$), sendo $b$ uma constante). Isso significa que, se você trocar a moeda de uma carteira, ou dobrar seu investimento em cada ação, seu risco aumentará na mesma proporção. Se você aposta 100 reais num jogo de poker e depois triplica a aposta, seu risco triplica também, pois você pode perder (ou ganhar) três vezes mais do que antes.</li>
</ol>

<h3> Importando bibliotecas </h3>

Para começar nosso projeto, vamos importar as bibliotecas necessárias.

In [6]:
import numpy as np #manipulação de matrizes/ álgebra linear
import pandas as pd #manipulação de dataframes
from datetime import datetime,timedelta #datas
import numpy.matlib
import matplotlib.pyplot as plt #imagens
import plotly.express as px #imagens interativas
import plotly.graph_objects as go
import pandas_datareader.data as web #importar dados históricos de ativos
import scipy.io #estatística
from scipy.optimize import linprog #algoritmo para otimização linear
from time import sleep 
from tqdm.notebook import tqdm #avaliar passar em um loop

<h3> Coletando o histórico de ações </h3>

Selecionaremos as principais ações da bolsa de valores do Brasil.

In [27]:
stocks = ['PETR4.SA','ABEV3.SA','CMIG4.SA','ITUB4.SA','VALE3.SA','BBDC4.SA','BBAS3.SA','GGBR4.SA','B3SA3.SA', 'MGLU3.SA',
         'MRVE3.SA','PRIO3.SA', 'CYRE3.SA', 'JHSF3.SA','WEGE3.SA', 'ENEV3.SA', 'DMMO3.SA', 'CASH3.SA', 'CSNA3.SA', 'RENT3.SA',
         'AMER3.SA']
start = datetime(2021,1,1)
end = datetime(2022,9,1)

In [28]:
historico_stocks = web.DataReader(stocks, 'yahoo', start,end)['Adj Close']

In [29]:
#colunas que contém nan
colunas_nan = list(historico_stocks.iloc[:,list(historico_stocks.isna().any())].columns)
if colunas_nan!=[]:
    print(f'Os ativos {colunas_nan} contém dados faltantes, e portante não devem ser utilizadas no portfólio')
    historico_stocks = historico_stocks.drop(columns=colunas_nan)
    stocks = list(historico_stocks.columns)
else:
    stocks = list(historico_stocks.columns)

In [38]:
fig = go.Figure()
for stock in stocks:
    fig.add_trace(go.Scatter(x=historico_stocks.index, y=historico_stocks[stock],
                                     mode='lines',
                                     name=f'{stock}'))
fig.update_layout(title='Histórico de fechamento das ações ', 
                    xaxis_title='Período',
                    yaxis_title='Preço')
fig.show()

<h3> Calcula o retorno e risco das ações </h3>

<p> O principal aspecto da teoria do portfólio é que o risco individual de um ativo é diferente de seu risco na carteira, tornando a diversificação capaz de minimizar o risco não-sistemático dos ativos em conjunto. Com a minimização, é possível escolher a proporção ideal de cada ativo no portfólio, otimizando a relação retorno/risco da carteira de títulos. A figura abaixo representa bem essa ideia: Para mais de 30 ativos, é possível mitigar praticamente todo o risco não-sistemático da carteira. O resto é risco de mercado, crédito, liquidez ou operacional. </p>

<img src="https://media-exp1.licdn.com/dms/image/C4D12AQHx1Wy9yxJRlw/article-inline_image-shrink_1000_1488/0/1559501707529?e=1629936000&v=beta&t=ZyiAkVi7HhjbCx_dU-q1Q6A2AITVAudFLkWk1Iuvyn0">

In [39]:
data = historico_stocks.values.transpose()
Returns = np.zeros((data.shape[1]-1,data.shape[0]))
for i in range(Returns.shape[1]):
    Returns[:,i] = np.log(data[i,1::]/data[i,0:-1])+1
Returns = Returns[:,:-1]

In [40]:
wE = 221 ; #Janela de estimativa
R = Returns[::-1,:] #(a:k:b) a = primeiro indice, k = step size, b = último
R = R[0:wE+1,:]

O retorno esperado de um ativo é dado por 
$$E(R) = \frac{1}{n}\sum_{i=1}^{n}R_{t},$$
onde $n$ é o tamanho do período histórico considerado e $R_{t}$ o retorno em cada instante $t$ do histórico. O retorno da
carteira $\bar{R}_{c}$, é média dos retornos esperados de cada ativo.

In [33]:
ExpR = np.mean(R, axis=0).reshape(-1,1).T #retorno esperado
MeanR = numpy.matlib.repmat(ExpR,wE+1,1)

<h3> Minimiza o VaR e CVaR </h3>

<p> Aqui a ideia é entender o risco como o quanto você aceita perder. Muito simples não? Suponha que você vai fazer um investimento de $1000$ reais. Seu gerente lhe diz que na carteira $X$ você pode ter um retorno de $300$% no ano e que você pode perder no máximo $900$ reais com chance de $5$%. O VaR é a perda máxima esperada (não confundir com a perda máxima possível), os $900$ reais, e o alfa do VaR é a chance de você perder mais que isso (no exemplo $5$%). Minimizar o VaR significa escolher o melhor conjunto de ativos que, com um mesmo retorno, diminua essa perda máxima esperada. </P>

<p> Como já mencionei, o VaR falha na subaditividade. E pior, falha numa propriedade que a galera da otimização adora (eu também!), a convexidade. Felizmente, temos uma medida que, além de ser convexa, é coerente. O Valor em risco condicional (CVaR) examina as perdas que excedem o limite do Valor em Risco (VaR). No exemplo que demos da carteira $X$, isso significa analisar as perdas para 5%, 4%,... de chance e tirar uma média disso. O VaR e o CVaR estão intimamente relacionados e, ao minimizar o CVaR, também levará a uma redução do VaR da carteira. A figura abaixo expressa uma curva normal com as perdas esperadas do Var e CVaR e as probabilidades esperadas. </p>

<ol>
<li>O VaR tenta resumir em um único número ($\alpha$), a perda máxima esperada dentro de um certo prazo com um certo grau de confiança estatística: $$VaR_{1-\alpha}(X):=inf\left \{t\in \mathbb{R} : Pr(X\leqslant  t)\geqslant 1-\alpha\right \}, \alpha\in [0,1]$$ </li>
<li>O CVaR pode ser definido como a esperança condicional de perdas das carteiras superiores ao VaR: 
 $$CVaR_{1-\alpha}(X):=inf\left (t+\frac{1}{\alpha}E[X-t]_{+} \right ), \alpha\in (0,1]$$ $$CVaR_{1-\alpha}(X)=\frac{1}{\alpha}\int_{0}^{\alpha}VaR_{1-t}(X)dt,$$ onde $[c]_{+}=max(0,c).$</li>

<img src="https://media-exp1.licdn.com/dms/image/C4D12AQFg17zgxgCBUQ/article-inline_image-shrink_1000_1488/0/1559505124297?e=1629936000&v=beta&t=AKHOCeCHUtinkk2eEB_lIlWH-6G1Stb4TC7BQxsvtWk">

A partir de uma série da manipulações matemáticas, Rockafellar e Uryasev (2000) reescreve o cálculo do CVaR em termos de uma função $F_{\beta}$, dada por
$$F_{\beta}(x,\alpha)=\alpha+(1-\beta)^{-1}\int_{y\in \mathbb{R}^{m}}[f(x,y)-\alpha]p(y)dy,$$ onde $p(y)$ é a função densidade de probabilidade de variáveis de mercado; $\beta$ é o nível de probabilidade a ser escolhido; e $f(x,y)$ é uma função de perda associada à carteira $x$ e a variáveis de mercado $y$.

Para o caso de valores discretos, a equação acima pode ser reescrita como:
$$F_{\beta}(x,\alpha)=\alpha+\frac{1}{n(1-\beta)}\sum_{k=1}^{n}[f(x,y)-\alpha]_{+}$$

Dessa maneira, Rockafellar e Uryasev (2000) usa a função $F_{\beta}$ linear para definir a forma para a otimização de uma carteira de ações utilizando o CVaR como medida de risco:
$$\begin{align*}
 Min\;\; \alpha+\frac{1}{n(1-\beta)}\sum_{k=1}^{n}[-w_{i}R_{i}-\alpha]_{+}
 \\ 
 S.a\;\; \sum_{k=1}^{n}w_{i}=1 \\ 
 0\leqslant w_{i}\leq 1 ,
\end{align*}$$
one $n$ é o tamanho da amostra e $w_{i}$ a proporção de cada ativo na carteira.

In [34]:
N = ExpR.shape[1] #número de ativos

nS = Returns.shape[0] #número de cenários
V0 = 1 #Initial wealth
a = 0.95 #Confidence level
h = 21 #Número de day trades por mês
meanR = np.linspace(np.min(ExpR),np.max(ExpR),50)
#Allocation for linprog output
linMap = np.zeros((nS+N+1,len(meanR)))
f = np.vstack((np.zeros((N,1)),1,1/((1-a)*nS)*np.ones((nS,1))))
#Coeficientes da restrição de desigualdade
w = np.ones((nS,N)) - Returns
v = -np.ones((nS,1))
y = -np.eye(nS)
A = np.hstack([w,v,y])
b = np.zeros((nS,1))
#Restrições do coeficiente de igualdade
Aeq = np.vstack([np.hstack([ExpR,np.array([0]).reshape(-1,1),np.zeros((1,nS))]),np.hstack([np.ones((1,N)),np.array([0]).reshape(-1,1),np.zeros((1,nS))])])
#Restrições de fronteira
lb = np.zeros((1,nS+N+1))
ub = np.full((1,nS+N+1), np.inf)
bounds = np.vstack([lb,ub]).transpose()
    
for i in tqdm(range(len(meanR))):
    beq = np.vstack([meanR[i]*V0,V0])
    linMap[:,i] = linprog(f,A,b,Aeq,beq,bounds, method='revised simplex')['x']          
VaR = linMap[N,:]
CVaR = sum(linMap[N+1::,:])/((1-a)*nS) + linMap[N,:]
w = linMap[0:ExpR.shape[1],np.argmin(CVaR)::]
meanR = 100*(meanR-1)*h
VaR = 100*VaR*np.sqrt(h)
CVaR = 100*CVaR*np.sqrt(h)



  0%|          | 0/50 [00:00<?, ?it/s]

Os gráficos abaixo expressam o retorno para cada valor em risco considerado. Devido ao fato da otimização do VaR não ser convexa, a fronteira eficiente da otimização com o VaR tem um comportamento mais "caótico" comparada à otimização com o CVaR, que tem um comportamento mais suave.

In [36]:
fig_1 = go.Figure(data=go.Scatter(x=VaR[meanR>0], y=meanR[meanR>0]))
fig_1.update_layout(title='Value at Risk - Fronteira Eficiente', 
    xaxis_title='Value at Risk (%)',
    yaxis_title='Retorno Esperado (%)')
fig_1.show()

In [22]:
fig_2 = go.Figure(data=go.Scatter(x=CVaR[meanR>0], y=meanR[meanR>0]))
fig_2.update_layout(title='Conditional Value at Risk - Fronteira Eficiente', 
                    xaxis_title='Conditional Value at Risk (%)',
                    yaxis_title='Retorno Esperado (%)')
fig_2.show()

O gráfico de área abaixo expressa a proporção de ativos na carteira para cada valor em risco considerado.

In [41]:
x=CVaR[np.argmin(CVaR)::]
y = 100*w

fig_2 = go.Figure()
for i in range(len(y)):
    fig_2.add_trace(go.Scatter(
                    x=x, y=y[i],
                    mode='lines',
                    line=dict(width=0.5),
                    name=stocks[i],
                    stackgroup='one'))
fig_2.update_layout(title='Distribuição de Ações na carteira - CVAR', 
                    xaxis_title='Conditional Value at Risk (%)',
                    yaxis_title='Distribuição (%)')
fig_2.show()

In [42]:
def proportion_CVAR(CVAR_value, CVaR_portfolio, y, stocks):
    i = np.argmin(abs(CVaR_portfolio-CVAR_value))
    CVAR = {}
    for j, stock in zip(range(len(stocks)-1), stocks):
        CVAR[stock] = round(y[j][i],2)
    CVAR = pd.Series(CVAR)
    CVAR = CVAR[CVAR!=0].sort_values()
    fig = px.bar(CVAR, y=CVAR.index,x=CVAR, orientation='h')
    fig.update_layout(
    title='Proporção de ativos na carteira',
    xaxis_title='Proporção',
    yaxis_title= 'Ações')
    return CVAR, fig

O resultado final é, para um dado valor em risco, as ações que devo investir e em qual proporção, para o meu portfólio ótimo.

In [43]:
proportion_CVAR(20, x, y, stocks)[1]

<h3> Backtest </h3>

O objetivo do backtest é analisar quais seriam os resultados de um carteira ótima criada num período anterior ao atual, verificando a evolução da carteira desde o instante em que ela foi criada, até o instante atual.

In [47]:
def Portfólio_Eficiente(ativos, data_inicio, data_fim, janela_estimativa=221, medida_risco='CVaR'):
    #medida_risco: ['VaR', 'CVaR']
    historico_stocks = web.DataReader(ativos, 'yahoo', data_inicio, data_fim)['Adj Close']
    #colunas que contém nan
    colunas_nan = list(historico_stocks.iloc[:,list(historico_stocks.isna().any())].columns)
    if colunas_nan!=[]:
        print(f'Os ativos {colunas_nan} contém dados faltantes, e portante não devem ser utilizadas no portfólio')
        historico_stocks = historico_stocks.drop(columns=colunas_nan)
        stocks = list(historico_stocks.columns)
    else:
        stocks = list(historico_stocks.columns)
        
    data = historico_stocks.values.transpose()
    Returns = np.zeros((data.shape[1]-1,data.shape[0]))
    for i in range(Returns.shape[1]):
        Returns[:,i] = np.log(data[i,1::]/data[i,0:-1])+1
    Returns = Returns[:,:-1]
    
    wE = janela_estimativa ; #Janela de estimativa
    a = 0.94 ; #parâmetro do modelo
    N = Returns.shape[1] ; #número de ações
    x= np.array([a**i for i in reversed(range(1,wE+2))]).reshape(-1,1)
    A = numpy.matlib.repmat(x, 1, N)
    R = Returns[::-1,:] #(a:k:b) a = primeiro indice, k = step size, b = último
    R = R[0:wE+1,:]
    ExpR = np.mean(R, axis=0).reshape(-1,1).T #retorno esperado
    MeanR = numpy.matlib.repmat(ExpR,wE+1,1)
    #risco da carteira
    
    N = ExpR.shape[1] #número de ativos
    nS = Returns.shape[0] #número de cenários
    V0 = 1 #Initial wealth
    a = 0.95 #Confidence level
    h = 21 #Número de day trades por mês
    meanR = np.linspace(np.min(ExpR),np.max(ExpR),50)
    #Allocation for linprog output
    linMap = np.zeros((nS+N+1,len(meanR)))
    f = np.vstack((np.zeros((N,1)),1,1/((1-a)*nS)*np.ones((nS,1))))
    #Coeficientes da restrição de desigualdade
    w = np.ones((nS,N)) - Returns
    v = -np.ones((nS,1))
    y = -np.eye(nS)
    A = np.hstack([w,v,y])
    b = np.zeros((nS,1))
    #Restrições do coeficiente de igualdade
    Aeq = np.vstack([np.hstack([ExpR,np.array([0]).reshape(-1,1),np.zeros((1,nS))]),np.hstack([np.ones((1,N)),np.array([0]).reshape(-1,1),np.zeros((1,nS))])])
    #Restrições de fronteira
    lb = np.zeros((1,nS+N+1))
    ub = np.full((1,nS+N+1), np.inf)
    bounds = np.vstack([lb,ub]).transpose()

    for i in tqdm(range(len(meanR))):
        beq = np.vstack([meanR[i]*V0,V0])
        linMap[:,i] = linprog(f,A,b,Aeq,beq,bounds, method='revised simplex')['x']        
    VaR = linMap[N,:]
    CVaR = sum(linMap[N+1::,:])/((1-a)*nS) + linMap[N,:]
    w_cvar = linMap[0:ExpR.shape[1],np.argmin(CVaR)::]
    w_var = linMap[0:ExpR.shape[1],np.argmin(VaR)::]
    meanR = 100*(meanR-1)*h
    VaR = 100*VaR*np.sqrt(h)
    CVaR = 100*CVaR*np.sqrt(h)
    
    x_cvar = CVaR[np.argmin(CVaR)::]
    x_var = VaR[np.argmin(VaR)::]
    y_cvar = 100*w_cvar
    y_var = 100*w_var
    if medida_risco=='CVaR':
        return x_cvar, y_cvar, stocks
    elif medida_risco=='VaR':
        return x_var, y_var, stocks

In [48]:
def BackTest(data_inicio, data_fim, portfolios, valor_investido, valor_risco):
    fig = go.Figure()
    for portfolio, risco in zip(portfolios, valor_risco):
        proporcao = list(portfolio.index)
        historico_stocks = web.DataReader(list(portfolio.index), 'yahoo', data_inicio, data_fim)['Adj Close']
        inv_inicio = historico_stocks.iloc[0]
        inv_fim = historico_stocks.iloc[-1]
        valor_atual = sum(portfolio/inv_inicio*inv_fim)/100*valor_investido
        print(f'Investimento inicial {valor_investido}')
        print(f'Para um risco = {risco}, o valor atual do investimento é {round(valor_atual,2)}')
    
        valor = {}
        for i in historico_stocks.index:
            inv_fim = historico_stocks.loc[i]
            v = sum(portfolio/inv_inicio*inv_fim)/100*valor_investido
            valor[i] = v
        valor = pd.Series(valor)
        fig.add_trace(go.Scatter(x=valor.index, y=valor,
                            mode='lines',
                            name=f'Risco = {risco}'))
    fig.update_layout(
    title='Evolução do Valor da Carteira', xaxis_title='Período',
    yaxis_title= 'Valor')
    fig.show()

In [50]:
acoes = ['PETR4.SA','ABEV3.SA','CMIG4.SA','ITUB4.SA','VALE3.SA','BBDC4.SA','BBAS3.SA','GGBR4.SA','B3SA3.SA', 'MGLU3.SA',
         'MRVE3.SA','PRIO3.SA', 'CYRE3.SA', 'JHSF3.SA','WEGE3.SA', 'ENEV3.SA', 'DMMO3.SA', 'CASH3.SA', 'CSNA3.SA', 'RENT3.SA',
         'AMER3.SA']
start = datetime(2021,1,1)
end = datetime(2022,9,1)
VaR_portfolio = Portfólio_Eficiente(acoes, start, end, janela_estimativa=100, medida_risco='VaR')
CVaR_portfolio = Portfólio_Eficiente(acoes, start, end, janela_estimativa=100, medida_risco='CVaR')

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [51]:
valor_risco = 20
x_cvar = CVaR_portfolio[0]
y_cvar = CVaR_portfolio[1]
stocks_cvar = CVaR_portfolio[2]

x_var = VaR_portfolio[0]
y_var = VaR_portfolio[1]
stocks_var = VaR_portfolio[2]

In [52]:
portfolio_cvar = proportion_CVAR(valor_risco, x_cvar, y_cvar, stocks_cvar)[0]
portfolio_cvar

CMIG4.SA     7.87
DMMO3.SA    14.64
BBAS3.SA    36.41
PETR4.SA    41.09
dtype: float64

In [53]:
portfolio_var = proportion_CVAR(valor_risco, x_var, y_var, stocks_var)[0]
portfolio_var

PETR4.SA    100.0
dtype: float64

In [55]:
data_inicio_backtest = end
data_fim_backtest = data_inicio_backtest+timedelta(days=60)
valor_cvar = range(5,30,5)
portfolios = [proportion_CVAR(risco, x_cvar, y_cvar, stocks_cvar)[0] for risco in valor_cvar]
BackTest(data_inicio_backtest, data_fim_backtest, portfolios, valor_investido=5000, valor_risco=valor_cvar)

Investimento inicial 5000
Para um risco = 5, o valor atual do investimento é 5032.8
Investimento inicial 5000
Para um risco = 10, o valor atual do investimento é 5032.8
Investimento inicial 5000
Para um risco = 15, o valor atual do investimento é 5007.04
Investimento inicial 5000
Para um risco = 20, o valor atual do investimento é 4957.44
Investimento inicial 5000
Para um risco = 25, o valor atual do investimento é 4918.15


In [56]:
data_inicio_backtest = end
data_fim_backtest = data_inicio_backtest+timedelta(days=60)
valor_var = range(5,30,5)
portfolios = [proportion_CVAR(risco, x_var, y_var, stocks_var)[0] for risco in valor_cvar]
BackTest(data_inicio_backtest, data_fim_backtest, portfolios, valor_investido=5000, valor_risco=valor_var)

NameError: name 'data_fim' is not defined